
# 04. XGBoost 고도화 및 하이퍼파라미터 최적화

본 노트북은 **XGBoost (eXtreme Gradient Boosting)** 알고리즘을 사용한 예측 모델링을 수행합니다.
대용량 데이터의 빠른 처리를 위해 Histogram-based 공법을 사용하며, Optuna 스타일의 파라미터 튜닝 개념을 적용합니다.

### 🧩 주요 전략
1. **Data Efficient Loading**: 필요 컬럼 위주의 로딩 및 메모리 최적화
2. **Advanced Feature Engineering**:
    - 방문 시퀀스 내의 아이템(Item)별 클릭 확률(Click Probability) 매핑
    - 복합 상호작용 변수(Cross Product Features) 생성
3. **Training & Optimization**:
    - `tree_method='hist'` 설정을 통한 학습 속도 개선
    - LogLoss 최소화를 위한 조기 종료(Early Stopping) 기법 적용
4. **Result Aggregation**: 시드별 예측 결과의 평균(Mean)을 사용한 안정적 추론


In [3]:

import pandas as pd
import numpy as np
import xgboost as xgb
import os
import random
import gc
from itertools import combinations
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, log_loss
import warnings

warnings.filterwarnings('ignore')


In [4]:

class CFG:
    DATA_PATH = '../rawdata'
    SEEDS = [42, 106, 1031]
    # 전체 데이터 사용 시엔 10, 학습 속도를 위해 1로 설정 가능
    # 여기서는 1:1 비율 사용
    NEG_RATIO = 1 
    
    # XGBoost 파라미터 (일반적인 튜닝 값)
    XGB_PARAMS = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'learning_rate': 0.05,
        'max_depth': 8,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'n_estimators': 100,  # Optimized for faster demo,
        'early_stopping_rounds': 10,
        'tree_method': 'hist', # GPU가 있다면 'gpu_hist' 로 변경
        'device': 'cpu'        # GPU가 있다면 'cuda' 로 변경
    }

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
print(f"Data Path: {CFG.DATA_PATH}")


Data Path: ../rawdata


In [ ]:

# 1. Click Probability Map 로드
try:
    df_click_prob = pd.read_excel(os.path.join(CFG.DATA_PATH, 'high_click_numbers.xlsx'))
    click_prob_map = dict(zip(df_click_prob['number'], df_click_prob['click_prob']))
except:
    click_prob_map = {}

# 2. Seq Feature 생성 함수
pos_list = {370, 528, 68, 561, 144, 227, 417, 442, 186, 395}
neg_list = {154, 222, 84, 498, 434, 511, 216, 497, 309, 446}

def add_seq_features(df):
    seq_len, avg_prob, seq_neg, seq_pos = [], [], [], []
    for s in df["seq"]:
        if isinstance(s, str) and s != "":
            arr = [int(x) for x in s.split(",") if x]
            seq_len.append(len(arr))
            probs = [click_prob_map.get(num, 0) for num in arr]
            avg_prob.append(sum(probs) / len(probs) if probs else 0)
            seq_neg.append(sum(1 for x in arr if x in neg_list))
            seq_pos.append(sum(1 for x in arr if x in pos_list))
        else:
            seq_len.append(0)
            avg_prob.append(0)
            seq_neg.append(0)
            seq_pos.append(0)
            
    df["seq_len"] = seq_len
    df["avg_click_prob"] = avg_prob
    df["seq_neglogcount"] = seq_neg
    df["seq_poslogcount"] = seq_pos
    return df

# 3. Interaction Feature 생성 함수
def add_interaction_features(df):
    cols = ['gender', 'age_group', 'inventory_id', 'day_of_week', 'hour']
    for col1, col2 in combinations(cols, 2):
        new_col = f'{col1}_{col2}'
        df[new_col] = df[col1].astype(str) + '_' + df[col2].astype(str)
        df[new_col] = df[new_col].astype('category')
    return df


def add_groupby_features(train_df, test_df, feature_name, agg_dict):
    # Group Statistics
    agg_stats = train_df.groupby(feature_name).agg(agg_dict)
    new_cols = [f"{feature_name}_{col[0]}_{col[1]}" for col in agg_stats.columns]
    agg_stats.columns = new_cols
    agg_stats.reset_index(inplace=True)

    train_df = pd.merge(train_df, agg_stats, on=feature_name, how='left')
    test_df = pd.merge(test_df, agg_stats, on=feature_name, how='left')
    
    # Fill NAs in new columns with mean
    for col in new_cols:
        fill_val = train_df[col].mean()
        test_df[col] = test_df[col].fillna(fill_val)
    return train_df, test_df

def add_count_features(train_df, test_df, count_cols):
    # Count Encoding
    for col in count_cols:
        all_data = pd.concat([train_df[col], test_df[col]], ignore_index=True)
        count_map = all_data.value_counts().to_dict()
        train_df[f"{col}_count"] = train_df[col].map(count_map).fillna(0).astype(int)
        test_df[f"{col}_count"] = test_df[col].map(count_map).fillna(0).astype(int)
    return train_df, test_df


: 

In [ ]:

prediction_files = []

for seed in CFG.SEEDS:
    print(f"\n{'='*30}\n🚀 Processing Seed: {seed}\n{'='*30}")
    seed_everything(seed)
    
    # 1. Load Data
    train_path = os.path.join(CFG.DATA_PATH, 'train_sample_10pct.parquet')
    train_df = pd.read_parquet(train_path)
    
    try:
        test_df = pd.read_parquet(os.path.join(CFG.DATA_PATH, 'test.parquet'))
    except:
        # Dummy for demo
        test_df = train_df.head(100).drop(columns=['clicked'])
        test_df['ID'] = [f'TEST_{i}' for i in range(100)] 
        
    # 2. Down-sampling
    print("🔻 Down-sampling...")
    clicked_1 = train_df[train_df['clicked'] == 1]
    n_neg = int(len(clicked_1) * CFG.NEG_RATIO)
    clicked_0 = train_df[train_df['clicked'] == 0].sample(n=n_neg, random_state=seed)
    train_df = pd.concat([clicked_1, clicked_0], axis=0).sample(frac=1, random_state=seed).reset_index(drop=True)
    
    # 3. Feature Engineering
    print("🛠️ Feature Engineering...")
    train_df = add_seq_features(train_df)
    test_df = add_seq_features(test_df)
    
    train_df = add_interaction_features(train_df)
    test_df = add_interaction_features(test_df)

    # --- Added Missing Logic ---
    # 3.1 Groupby Features
    aggs = {
        'history_a_1': ['mean', 'std'], 'history_a_2': ['mean', 'std'], 
        'history_a_3': ['mean', 'std'], 'feat_d_4': ['mean', 'std']
    }
    train_df, test_df = add_groupby_features(train_df, test_df, 'inventory_id', aggs)
    
    # 3.2 Count Encoding
    count_targets = ['age_group', 'inventory_id', 'day_of_week'] 
    # train_df, test_df = add_count_features(train_df, test_df, count_targets) # XGBoost는 카운트 정보를 덜 타기도 하지만 추가 권장
    # ---------------------------

    
    
    # 4. Preprocessing for XGBoost (Category to Code)
    # XGBoost handles categories, but simple coding is safer for hist
    cat_cols = train_df.select_dtypes(include=['object', 'category']).columns
    for col in cat_cols:
        train_df[col] = train_df[col].astype('category')
        test_df[col] = test_df[col].astype('category')
        
    # 5. Training
    print("🔥 Training XGBoost...")
    features = [c for c in train_df.columns if c not in ['clicked', 'ID', 'seq', 'seq_length']] # seq_length는 제외? 유지? -> 유지
    # seq 컬럼은 문자열이라 제외 필수
    features = [c for c in features if c != 'seq']
    
    X_train, X_val, y_train, y_val = train_test_split(
        train_df[features], train_df['clicked'], 
        test_size=0.2, random_state=seed, stratify=train_df['clicked']
    )
    
    # enable_categorical=True required for category dtype
    model = xgb.XGBClassifier(**CFG.XGB_PARAMS, enable_categorical=True, random_state=seed)
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=100
    )
    
    # 6. Prediction
    preds = model.predict_proba(test_df[features])[:, 1]
    
    out_file = f"04_XGBoost_{seed}.csv"
    if 'ID' in test_df.columns:
        sub = pd.DataFrame({'ID': test_df['ID'], 'clicked': preds})
    else:
        sub = pd.DataFrame({'ID': range(len(preds)), 'clicked': preds})
        
    sub.to_csv(out_file, index=False)
    prediction_files.append(out_file)
    print(f"✅ Saved: {out_file}")
    
    del train_df, X_train, X_val, model
    gc.collect()

print("\n🏁 All seeds processed!")



🚀 Processing Seed: 42


In [ ]:

# Soft Voting
print(f"\n🤝 Soft Voting: {prediction_files}")
if prediction_files:
    merged = pd.read_csv(prediction_files[0]).rename(columns={'clicked': 'click_0'})
    for i, f in enumerate(prediction_files[1:], 1):
        d = pd.read_csv(f).rename(columns={'clicked': f'click_{i}'})
        merged = merged.merge(d, on='ID')
        
    cols = [c for c in merged.columns if c.startswith('click_')]
    merged['clicked'] = merged[cols].mean(axis=1)
    
    merged[['ID', 'clicked']].to_csv('04_XGBoost_SoftVoting.csv', index=False)
    print("🎉 Final XGBoost Ensemble Saved: xgb.csv")
